In [1]:
from pathlib import Path
import requests
import pandas as pd
import os

DATA_DIR = Path('..') / 'data'

In [2]:
def download_one_file_of_raw_data(year: int, month: int) -> Path:
    URL = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
    response = requests.get(URL)

    if response.status_code == 200:
        path = f"{DATA_DIR}/raw/rides_{year}_{month:02d}.parquet"
        open(path, "wb").write(response.content)
        return path
    else:
        raise Exception(f"{URL} is not available")

In [3]:
def validate_raw_data(year: int, month: int) -> Path:
    rides = pd.read_parquet(f"{DATA_DIR}/raw/rides_{year}_{month:02d}.parquet")
    rides = rides[['tpep_pickup_datetime', 'PULocationID']]
    rides.rename(columns={
        'tpep_pickup_datetime': 'pickup_datetime',
        'PULocationID': 'pickup_location_id'
    }, inplace=True)
    rides = rides[rides.pickup_datetime >= '2022-01-01']
    rides = rides[rides.pickup_datetime < '2022-02-01']
    transformed_dir = DATA_DIR / "transformed" 
    if not transformed_dir.exists():
        transformed_dir.mkdir(parents=True, exist_ok=True)
    path = transformed_dir / f"validated_rides_{year}_{month:02d}.parquet"
    rides.to_parquet(path)
    return rides

In [4]:
download_one_file_of_raw_data(2022, 1)
rides = clean_data(2022, 1)

In [5]:
rides['pickup_datetime'].describe()

count                          2463879
mean     2022-01-17 01:58:40.393673472
min                2022-01-01 00:00:08
25%                2022-01-09 15:37:56
50%                2022-01-17 12:11:54
75%                2022-01-24 13:49:37
max                2022-01-31 23:59:58
Name: pickup_datetime, dtype: object